In [1]:
import pandas as pd
import os
from tqdm.auto import tqdm
from pandarallel import pandarallel
from rdkit import Chem
from tqdm import tqdm as top_tqdm

In [2]:
def get_structure_sequence(pdb_file):
    try:
        mol = Chem.MolFromPDBFile(pdb_file)
        protein_sequence = Chem.MolToSequence(mol)
    except:
        protein_sequence = ''
    return protein_sequence

def multiprocess_structure_check(df, nb_workers, pdb_file_path):
    
    if nb_workers != 0:

        pandarallel.initialize(nb_workers=nb_workers, progress_bar=True)
        df['pdb_files'] = df['alphafolddb-id'].parallel_apply(
            lambda x: os.path.join(pdb_file_path, f'AF-{x}-F1-model_v4.pdb'))
        df['aa_sequence_calculated'] = df['pdb_files'].parallel_apply(
            lambda x: get_structure_sequence(x))
    else:
        top_tqdm.pandas(desc='pandas bar')
        df['pdb_files'] = df['alphafolddb-id'].progress_apply(
            lambda x: os.path.join(pdb_file_path, f'AF-{x}-F1-model_v4.pdb'))
        df['aa_sequence_calculated'] = df['pdb_files'].progress_apply(
            lambda x: get_structure_sequence(x))
    
    df['is_valid'] = (df['aa_sequence_calculated'] == df['aa_sequence'])

    return df


def get_blast_database(dir, fasta_path):
    database_df = pd.DataFrame()
    csv_fnames = os.listdir(dir)
    pbar = tqdm(
        csv_fnames,
        total=len(csv_fnames)
    )
    for fname in pbar:
        df = pd.read_csv(os.path.join(dir, fname))
        df = df[['alphafolddb-id', 'aa_sequence', 'site_labels', 'site_types']]
        database_df = pd.concat([database_df, df])
    
    database_df = database_df.drop_duplicates(subset=['alphafolddb-id', 'aa_sequence','site_labels', 'site_types']).reset_index(drop=True)
    database_df['alphafolddb-id'] = database_df['alphafolddb-id'].apply(lambda x:x.replace(';',''))

    with open(fasta_path, 'w', encoding='utf-8') as f:
        for idx, row in tqdm(database_df.iterrows(), total=len(database_df)):
            f.write('>{}\n'.format(row['alphafolddb-id']))
            f.write('{}\n'.format(row['aa_sequence']))
    return database_df

def get_query_database(path, fasta_path, pdb_file_path):
    database_df = pd.read_csv(path)
    database_df = database_df[['alphafolddb-id', 'aa_sequence','site_labels', 'site_types']]
    database_df['alphafolddb-id'] = database_df['alphafolddb-id'].apply(lambda x:x.replace(';',''))
    
      
    
    write_database_df = database_df.drop_duplicates(subset=['alphafolddb-id', 'aa_sequence','site_labels', 'site_types']).reset_index(drop=True)


    with open(fasta_path, 'w', encoding='utf-8') as f:
        for idx, row in tqdm(write_database_df.iterrows(), total=len(write_database_df)):
            f.write('>{}\n'.format(row['alphafolddb-id']))
            f.write('{}\n'.format(row['aa_sequence']))
    return database_df



           

In [3]:
dataset_path = '../../dataset/mcsa_fine_tune/normal_mcsa'
blast_database_df = get_blast_database(os.path.join(dataset_path, 'train_dataset'), fasta_path=os.path.join(dataset_path, 'blast_database.fasta'))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/771 [00:00<?, ?it/s]

In [4]:
test_dataset = get_query_database(os.path.join(dataset_path, 'test_dataset', 'mcsa_test.csv'), fasta_path=os.path.join(dataset_path, 'test_dataset.fasta'), pdb_file_path=os.path.join(os.path.dirname(dataset_path), 'structures', 'alphafolddb_download'))


  0%|          | 0/95 [00:00<?, ?it/s]

In [5]:
test_dataset = multiprocess_structure_check(test_dataset, 10, pdb_file_path='../../dataset/mcsa_fine_tune/structures/alphafolddb_download')
test_dataset = test_dataset.loc[test_dataset['is_valid']].reset_index(drop=True)
test_dataset

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


,alphafolddb-id,aa_sequence,site_labels,site_types,pdb_files,aa_sequence_calculated,is_valid
0,P07598,MSRTVMERIEYEMHTPDPKADPDKLHFVQIDEAKCIGCDTCSQYCP...,"[[156], [159], [178], [198], [237], [240], [24...",NaN,../../dataset/mcsa_fine_tune/structures/alphaf...,MSRTVMERIEYEMHTPDPKADPDKLHFVQIDEAKCIGCDTCSQYCP...,True
1,P00436,MPIELLPETPSQTAGPYVHIGLALEAAGNPTRDQEIWNRLAKPDAP...,"[[109], [148], [158], [161], [163]]",NaN,../../dataset/mcsa_fine_tune/structures/alphaf...,MPIELLPETPSQTAGPYVHIGLALEAAGNPTRDQEIWNRLAKPDAP...,True
2,Q55389,MTSSDTQNNKTLAAMKNFAEQYAKRTDTYFCSDLSVTAVVIEGLAR...,"[[56], [58], [75], [77], [86], [87], [88]]",NaN,../../dataset/mcsa_fine_tune/structures/alphaf...,MTSSDTQNNKTLAAMKNFAEQYAKRTDTYFCSDLSVTAVVIEGLAR...,True
3,P68688,MQTVIFGRSGCPYCVRAKDLAEKLSNERDDFQYQYVDIRAEGITKE...,"[[8], [10], [11], [13], [14], [18], [72]]",NaN,../../dataset/mcsa_fine_tune/structures/alphaf...,MQTVIFGRSGCPYCVRAKDLAEKLSNERDDFQYQYVDIRAEGITKE...,True
4,P0A006,MDKKTIYFICTGNSCRSQMAEGWGKEILGEGWNVYSAGIETHGVNP...,"[[10], [11], [12], [13], [14], [15], [16], [17...",NaN,../../dataset/mcsa_fine_tune/structures/alphaf...,MDKKTIYFICTGNSCRSQMAEGWGKEILGEGWNVYSAGIETHGVNP...,True
...,...,...,...,...,...,...,...
77,P42126,MALVASVRVPARVLLRAGARLPGAALGRTERAAGGGDGARRFGSQR...,"[[108], [153], [177], [178]]",NaN,../../dataset/mcsa_fine_tune/structures/alphaf...,MALVASVRVPARVLLRAGARLPGAALGRTERAAGGGDGARRFGSQR...,True
78,P27001,MLEEALAAIQNARDLEELKALKARYLGKKGLLTQEMKGLSALPLEE...,"[[149], [178], [204], [218], [261], [314]]",NaN,../../dataset/mcsa_fine_tune/structures/alphaf...,MLEEALAAIQNARDLEELKALKARYLGKKGLLTQEMKGLSALPLEE...,True
79,P22106,MCSIFGVFDIKTDAVELRKKALELSRLMRHRGPDWSGIYASDNAIL...,"[[2], [51], [75], [76], [322], [325]]",NaN,../../dataset/mcsa_fine_tune/structures/alphaf...,MCSIFGVFDIKTDAVELRKKALELSRLMRHRGPDWSGIYASDNAIL...,True
80,P46849,MKRMIALDGAQGEGGGQILRSALSLSMITGQPFTITSIRAGRAKPG...,"[[13], [308]]",NaN,../../dataset/mcsa_fine_tune/structures/alphaf...,MKRMIALDGAQGEGGGQILRSALSLSMITGQPFTITSIRAGRAKPG...,True


In [6]:
import subprocess

database_fasta = os.path.join(dataset_path, 'blast_database.fasta')
database = os.path.join(dataset_path, 'blast_database')
command = f'makeblastdb -in {database_fasta} -dbtype prot -out {database}'
subprocess.run(command, shell=True)



Building a new DB, current time: 05/23/2024 20:38:11
New DB name:   /home/xiaoruiwang/data/ubuntu_work_beta/single_step_work/EasIFA_v2/dataset/mcsa_fine_tune/normal_mcsa/blast_database
New DB title:  ../../dataset/mcsa_fine_tune/normal_mcsa/blast_database.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /home/xiaoruiwang/data/ubuntu_work_beta/single_step_work/EasIFA_v2/dataset/mcsa_fine_tune/normal_mcsa/blast_database
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 771 sequences in 0.0106261 seconds.


CompletedProcess(args='makeblastdb -in ../../dataset/mcsa_fine_tune/normal_mcsa/blast_database.fasta -dbtype prot -out ../../dataset/mcsa_fine_tune/normal_mcsa/blast_database', returncode=0)

In [7]:
query_file = os.path.join(dataset_path, 'test_dataset.fasta')
output_file = os.path.join(dataset_path, 'blast_results.txt')
command = f'blastp -query {query_file} -db {database} -out {output_file} -evalue 0.001 -outfmt 6'
if not os.path.exists(output_file):
    subprocess.run(command, shell=True)


In [8]:
def read_blast_results(path):
    column_headers = [
    "Query ID",
    "Subject ID",
    "% Identity",
    "Alignment Length",
    "Mismatches",
    "Gap Opens",
    "Query Start",
    "Query End",
    "Subject Start",
    "Subject End",
    "E-value",
    "Bit Score",
    ]
    results_df = pd.read_csv(path, sep='\t', header=None)
    results_df.columns = column_headers
    return results_df



In [9]:
blast_p_results = read_blast_results(path=output_file)
blast_p_results

,Query ID,Subject ID,% Identity,Alignment Length,Mismatches,Gap Opens,Query Start,Query End,Subject Start,Subject End,E-value,Bit Score
0,Q5SJ80,P0ABJ3,21.843,293,196,7,218,505,269,533,4.290000e-10,57.0
1,Q5SJ80,P0ABJ1,21.843,293,196,7,218,505,269,533,4.290000e-10,57.0
2,Q5SJ80,P0ABI8,21.843,293,196,7,218,505,269,533,4.290000e-10,57.0
3,Q9ZFQ5,P49050,21.500,200,133,5,11,186,606,805,4.210000e-05,40.0
4,P0A006,P11064,22.581,93,60,3,4,87,7,96,1.330000e-04,34.7
...,...,...,...,...,...,...,...,...,...,...,...,...
79,H2IFX0,P0AES2,21.719,221,147,7,156,357,194,407,3.850000e-06,43.1
80,P42126,P14604,22.932,266,192,7,40,298,25,284,3.300000e-12,60.8
81,P42126,Q62651,25.806,186,123,5,68,241,78,260,1.600000e-09,52.8
82,P42126,P52045,25.134,187,126,8,57,237,14,192,6.110000e-05,38.5


In [10]:
print(blast_p_results['% Identity'].max())
print(blast_p_results['% Identity'].min())
print(blast_p_results['% Identity'].mean())

46.885
21.5
27.482214285714296


In [11]:
import sys
sys.path.append('../../')
from dataset_preprocess.pdb_preprocess_utils import map_active_site_for_one
from utils import predict_activate_site_with_sequence_alignment, predict_activate_site_type_with_sequence_alignment

/home/xiaoruiwang/software/miniconda3/envs/py38/lib/python3.8/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [12]:
predicted_activate_sites, overlap_scores, false_positive_rates = predict_activate_site_with_sequence_alignment(test_dataset, database=blast_database_df, blastp_results=blast_p_results, top_n=5)

  0%|          | 0/82 [00:00<?, ?it/s]

Get 82 results
Accuracy: 0.9821, Precision: 0.1995, Specificity: 0.9974, Overlap Score: 0.1854, False Positive Rate: 0.0026, F1: 0.1812, MCC: 0.1838


In [13]:
# predicted_activate_sites, predicted_activate_sites_vec, overlap_scores_list, false_positive_rates_list = predict_activate_site_type_with_sequence_alignment(test_dataset, database=blast_database_df, blastp_results=blast_p_results, top_n=5)